In [2]:
:dep itertools
:dep ndarray
:dep ndarray-interp
:dep vawt = {path="../"}
:dep csv
:dep ndarray-csv
:dep plotters = {default_features = false, features = ["evcxr", "all_series"]}
:dep log
:dep env_logger

use std::fs::File;
use std::f64::consts::{FRAC_PI_2, PI};
use std::error::Error;

use itertools::Itertools;
use csv::ReaderBuilder;
use ndarray::{array, Array2, Axis, Array};
use ndarray_csv::Array2Reader;
use plotters::prelude::*;
use log::{LevelFilter, info, warn};
use env_logger::Env;

use vawt::areofoil::Aerofoil;
use vawt::{VAWTSolver, VAWTSolution};

In [3]:
env_logger::init();
log::set_max_level(LevelFilter::Info);

In [4]:
fn read_array(path: &str) -> Result<Array2<f64>, Box<dyn Error>> {
    let file = File::open(path)?;
    let mut reader = ReaderBuilder::new()
        .has_headers(false)
        .trim(csv::Trim::All)
        .delimiter(b',')
        .from_reader(file);

    let mut arr: Array2<f64> = reader.deserialize_array2_dynamic().unwrap();
    for mut datapoint in arr.axis_iter_mut(Axis(0)) {
        datapoint[0] = datapoint[0].to_radians();
    }
    Ok(arr)
}

In [5]:
let aerofoil = Aerofoil::builder()
    .add_data_row(read_array("NACA0018/NACA0018Re0080.data")?, 80_000.0)?
    .add_data_row(read_array("NACA0018/NACA0018Re0040.data")?, 40_000.0)?
    .add_data_row(read_array("NACA0018/NACA0018Re0160.data")?, 160_000.0)?
    .set_aspect_ratio(12.8)
    .update_aspect_ratio(true)
    .symmetric(true)
    .build()?;
let aerofoil: &'static Aerofoil = Box::leak(Box::new(aerofoil));
let n = 90;
let mut solver = VAWTSolver::new(&aerofoil);
solver.re(31_300.0)
    .solidity(0.3525)
    .n_streamtubes(n)
    .epsilon(0.0001);

In [12]:
let solution = solver.tsr(3.25).solve_optimize_beta();

let d_t_half = PI / n as f64;
let theta = Array::linspace(d_t_half, 2.0 * PI - d_t_half, n);

evcxr_figure((640 * 2 ,480), |root| {    
    let (left, right) = root.split_horizontally(640);

    left.titled("induction factor and c_tan", ("Arial", 20).into_font())?;
    // Create a chart context with the specified ranges
    let mut chart_left = ChartBuilder::on(&left)
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_cartesian_2d(0f64..360.0, -1f64..1f64)?;

    // Customize the x-axis labels
    chart_left.configure_mesh()
        .x_labels(10)
        .y_labels(10)
        .x_label_formatter(&|x| format!("{:.1}°", x))
        .draw()?;

    chart_left.draw_series(LineSeries::new(
        theta.iter().map( |&theta| (theta.to_degrees(), solution.a(theta))), 
        &RED
    ))?;
    chart_left.draw_series(LineSeries::new(
        theta.iter().map( |&theta| (theta.to_degrees(), solution.c_tan(theta))), 
        &BLUE
    ))?;

    right.titled("alpha and beta", ("Arial", 20).into_font())?;
    // Create a chart context with the specified ranges
    let mut chart_right = ChartBuilder::on(&right)
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_cartesian_2d(0f64..360.0, -20f64..20f64)?;

    // Customize the x-axis labels
    chart_right.configure_mesh()
        .x_labels(10)
        .y_labels(10)
        .x_label_formatter(&|x| format!("{:.1}°", x))
        .y_label_formatter(&|y| format!("{:.1}°", y))
        .draw()?;

    chart_right.draw_series(LineSeries::new(
        theta.iter().map(|&theta| (theta.to_degrees(), solution.beta(theta).to_degrees())),
        &GREEN
    ))?;

    chart_right.draw_series(LineSeries::new(
        theta.iter().map(|&theta| (theta.to_degrees(), solution.alpha(theta).to_degrees())),
        &BLUE
    ))?;
    Ok(())
})


induction factor and c_tan
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
-1.0
 
 
 
-0.5
 
 
 
0.0
 
 
 
0.5
 
 
 
1.0
 
 
 
 
0.0°
 
 
 
50.0°
 
 
 
100.0°
 
 
 
150.0°
 
 
 
200.0°
 
 
 
250.0°
 
 
 
300.0°
 
 
 
350.0°
 
 
<polyline fill="none" opacity="1" stroke="#FF0000" stroke-width="1" points="43,148 49,137 56,177 63,181 69,178 76,178 83,178 89,176 96,178 103,178 109,179 116,180 123,181 129,182 136,182 143,183 149,184 156,183 163,183 169,183 176,182 183,183 189,183 196,184 203,183 209,184 216,185 223,186 229,187 236,187 242,188 249,188 256,189 262,189 269,191 276,192 282,193 289,194 296,193 302,194 309,196 316,199 322,202 329,209 336,231 342,226 349,206 356,189 362,181 369,174 376,168 382,165 389,163 396,159 402,158 409,156 416,157 422,157 429,156 436,154 442,154 449,152 455,153 462,152 469,152 475,153 482,151 489,152 495,152 502,151 509,150 515,150 522,150 529,149 535,147 542,149 549,149 555,148 562,149 569,148 575,149 582,149 589,149 595,148 602,146 609,144 615,144 622,119 629,0 635,0 "/>
<polyline fill="none" opacity="1" stroke="#0000FF" stroke-width="1" points="43,223 49,223 56,222 63,221 69,219 76,218 83,217 89,216 96,214 103,213 109,212 116,211 123,210 129,209 136,208 143,207 149,206 156,205 163,204 169,203 176,202 183,201 189,200 196,200 203,199 209,199 216,198 223,198 229,198 236,198 242,199 249,199 256,200 262,201 269,202 276,203 282,205 289,207 296,209 302,211 309,213 316,216 322,219 329,221 336,224 342,223 349,222 356,220 362,219 369,218 376,217 382,216 389,215 396,214 402,214 409,213 416,213 422,213 429,213 436,213 442,213 449,213 455,213 462,213 469,214 475,214 482,214 489,215 495,215 502,215 509,215 515,216 522,216 529,216 535,217 542,217 549,218 555,218 562,219 569,220 575,220 582,220 589,221 595,221 602,222 609,222 615,222 622,223 629,224 635,223 "/>
 
alpha and beta
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
-20.0°
 
 
 
-15.0°
 
 
 
-10.0°
 
 
 
-5.0°
 
 
 
0.0°
 
 
 
5.0°
 
 
 
10.0°
 
 
 
15.0°
 
 
 
20.0°
 
 
 
 
0.0°
 
 
 
50.0°
 
 
 
100.0°
 
 
 
150.0°
 
 
 
200.0°
 
 
 
250.0°
 
 
 
300.0°
 
 
 
350.0°
 
 
<polyline fill="none" opacity="1" stroke="#00FF00" stroke-width="1" points="683,211 689,177 696,204 703,212 709,216 716,223 723,230 729,234 736,243 743,250 749,258 756,266 763,274 769,282 776,289 783,295 789,302 796,306 803,309 809,312 816,314 823,318 829,319 836,323 843,321 849,324 856,323 863,323 869,323 876,320 882,317 889,311 896,307 902,299 909,293 916,285 922,277 929,266 936,250 942,237 949,225 956,215 962,201 969,198 976,218 982,233 989,247 996,251 1002,244 1009,238 1016,232 1022,224 1029,217 1036,214 1042,206 1049,201 1056,194 1062,189 1069,185 1076,182 1082,179 1089,178 1095,175 1102,174 1109,172 1115,171 1122,172 1129,172 1135,172 1142,174 1149,175 1155,177 1162,179 1169,182 1175,186 1182,187 1189,191 1195,194 1202,197 1209,201 1215,203 1222,207 1229,210 1235,215 1242,220 1249,224 1255,228 1262,235 1269,232 1275,222 "/>
<polyline fill="none" opacity="1" stroke="#0000FF" stroke-width="1" points="683,233 689,274 696,258 703,259 709,263 716,265 723,266 729,270 736,269 743,270 749,270 756,270 763,270 769,270 776,270 783,270 789,270 796,271 803,273 809,275 816,276 823,276 829,278 836,278 843,281 849,280 856,281 863,281 869,281 876,282 882,282 889,284 896,284 902,285 909,284 916,284 922,284 929,284 936,288 942,288 949,286 956,283 962,282 969,269 976,233 982,195 989,170 996,158 1002,157 1009,157 1016,157 1022,159 1029,158 1036,158 1042,161 1049,163 1056,166 1062,167 1069,169 1076,170

In [7]:
let tsr = Array::linspace(1.0,5.0,30);
let cps = Array::from_iter(
    tsr.iter().map(|&tsr| {
        solver.tsr(tsr).solve_with_beta(0.0).c_power()
    })
);
let cps_opt = Array::from_iter(
    tsr.iter().map(|&tsr| {
        solver.tsr(tsr).solve_with_beta_fn(|theta| solution.beta(theta)).c_power()
    })
);

In [8]:
evcxr_figure((640,480), |root| {    
    // Create a chart context with the specified ranges
    let mut chart = ChartBuilder::on(&root)
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_cartesian_2d(1f64..5f64, 0f64..0.6)?;

    // Customize the x-axis labels
    chart.configure_mesh()
        .x_labels(8)
        .y_labels(5)
        .draw()?;

    chart.draw_series(LineSeries::new(
        tsr.iter().cloned().zip(cps.iter().cloned()), 
        &RED
    ))?;
    chart.draw_series(LineSeries::new(
        tsr.iter().cloned().zip(cps_opt.iter().cloned()),
        &BLUE
    ))?;
    Ok(())
})

0.0
 
 
 
0.2
 
 
 
0.4
 
 
 
0.6
 
 
 
 
1.0
 
 
 
2.0
 
 
 
3.0
 
 
 
4.0
 
 
 
5.0